In [1]:
import numpy as np
from pathlib import Path
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix

DATA_DIR = Path("data/eclss_preprocessed")

# Load scores and labels
recon = np.load(DATA_DIR / "vae_test_recon_errors.npy")
y_bin = np.load(DATA_DIR / "y_test_all_binary.npy")  # 0=nominal, 1=anomaly

print("AUC:", roc_auc_score(y_bin, recon))

# Sweep all thresholds from ROC curve
fpr, tpr, thresholds = roc_curve(y_bin, recon)

best_idx = np.argmax(tpr - fpr)  # Youden's J = tpr - fpr
best_thr = thresholds[best_idx]
print(f"Best threshold by Youden's J: {best_thr:.5f}")
print(f"  At this threshold: FPR={fpr[best_idx]:.3f}, TPR={tpr[best_idx]:.3f}")

# Evaluate at this threshold
y_pred = (recon >= best_thr).astype(int)
cm = confusion_matrix(y_bin, y_pred)
print("\nConfusion matrix at best threshold:")
print(cm)
print("\nClassification report:")
print(classification_report(y_bin, y_pred, target_names=["Nominal", "Anomaly"]))


AUC: 0.8490740740740741
Best threshold by Youden's J: 0.16803
  At this threshold: FPR=0.111, TPR=0.828

Confusion matrix at best threshold:
[[ 16   2]
 [ 31 149]]

Classification report:
              precision    recall  f1-score   support

     Nominal       0.34      0.89      0.49        18
     Anomaly       0.99      0.83      0.90       180

    accuracy                           0.83       198
   macro avg       0.66      0.86      0.70       198
weighted avg       0.93      0.83      0.86       198



In [40]:
"""
analyze_vae_eclss.py

Post-hoc analysis for the Dense VAE trained on the ECLSS synthetic dataset.

This script:
  1) Loads:
       - vae_test_recon_errors.npy  (shape: [N_test])
       - vae_test_latent_mu.npy      (shape: [N_test, latent_dim])
       - y_test_all_binary.npy      (0=nominal, 1=anomaly)
       - y_test_all_sys.npy         (system-level labels 0..5)
  2) Computes ROC curve and AUC for reconstruction error.
  3) Finds a good threshold using Youden's J statistic.
  4) Prints confusion matrix + classification report at that threshold.
  5) Generates:
       - ROC curve plot
       - Histogram of reconstruction errors (nominal vs anomaly)
       - 2D t-SNE of latent space colored by system fault class
"""

from __future__ import annotations

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve,
    auc,
    confusion_matrix,
    classification_report,
)
from sklearn.manifold import TSNE

# -----------------------------------------------------------
# PATHS
# -----------------------------------------------------------

try:
    REPO_ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # Notebook / interactive: assume CWD is repo root
    REPO_ROOT = Path.cwd()

DATA_ROOT = REPO_ROOT / "data"
PREP_DIR = DATA_ROOT / "eclss_preprocessed"
PREP_DIR.mkdir(parents=True, exist_ok=True)

# Files produced by train_vae_dense_eclss.py
RECON_FILE = PREP_DIR / "vae_test_recon_errors.npy"
LATENT_FILE = PREP_DIR / "vae_test_latent_mu.npy"
Y_BIN_FILE = PREP_DIR / "y_test_all_binary.npy"
Y_SYS_FILE = PREP_DIR / "y_test_all_sys.npy"

OUT_DIR = REPO_ROOT / "figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)


# -----------------------------------------------------------
# 1) LOAD DATA
# -----------------------------------------------------------

print("============================================================")
print(" ANALYZING DENSE VAE – ECLSS SYNTHETIC DATASET")
print("============================================================")
print(f"Using preprocessed dir : {PREP_DIR}")
print(f"Saving figures to       : {OUT_DIR}\n")

recon_errors = np.load(RECON_FILE)  # shape (N_test,)
z_mu = np.load(LATENT_FILE)         # shape (N_test, latent_dim)
y_bin = np.load(Y_BIN_FILE)         # shape (N_test,)
y_sys = np.load(Y_SYS_FILE)         # shape (N_test,)

N = len(recon_errors)
print(f"N_test = {N}")
print(f"  Nominal samples: {(y_bin == 0).sum()}")
print(f"  Anomaly samples: {(y_bin == 1).sum()}\n")


# -----------------------------------------------------------
# 2) ROC CURVE + AUC
# -----------------------------------------------------------

# Higher reconstruction error = "more anomalous".
# So we use recon_errors directly as the score.
fpr, tpr, thresholds = roc_curve(y_bin, recon_errors)
roc_auc = auc(fpr, tpr)

print(f"AUC (reconstruction error vs. binary label): {roc_auc:.4f}")

# -----------------------------------------------------------
# 3) BEST THRESHOLD BY YOUDEN'S J
# -----------------------------------------------------------

# Avoid the first threshold = inf (sklearn convention)
# by starting from index 1.
youden_j = tpr - fpr
idx_best = np.argmax(youden_j)
best_thr = thresholds[idx_best]
best_fpr = fpr[idx_best]
best_tpr = tpr[idx_best]

print(f"Best threshold by Youden's J: {best_thr:.5f}")
print(f"  At this threshold: FPR={best_fpr:.3f}, TPR={best_tpr:.3f}\n")

# Predictions at best threshold
y_pred_best = (recon_errors >= best_thr).astype(int)

cm = confusion_matrix(y_bin, y_pred_best)
print("Confusion matrix at best threshold:")
print(cm, "\n")

print("Classification report:")
print(classification_report(y_bin, y_pred_best, target_names=["Nominal", "Anomaly"]))


# -----------------------------------------------------------
# 4) PLOTS
# -----------------------------------------------------------

# 4.1 ROC curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, label=f"ROC (AUC = {roc_auc:.3f})")
plt.plot([0, 1], [0, 1], "k--", label="Random")
plt.scatter(best_fpr, best_tpr, color="red", label=f"Best J (τ={best_thr:.3f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – VAE Reconstruction Error")
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
roc_path = OUT_DIR / "vae_roc_recon_error.png"
plt.savefig(roc_path, dpi=300)
plt.close()
print(f"✅ Saved ROC curve to: {roc_path}")

# 4.2 Histogram of reconstruction errors (nominal vs anomaly)
plt.figure(figsize=(7, 4))
plt.hist(
    recon_errors[y_bin == 0],
    bins=20,
    alpha=0.6,
    label="Nominal",
    density=False,
)
plt.hist(
    recon_errors[y_bin == 1],
    bins=20,
    alpha=0.6,
    label="Anomaly",
    density=False,
)
plt.axvline(best_thr, color="red", linestyle="--", label=f"Thresh = {best_thr:.3f}")
plt.xlabel("Reconstruction error")
plt.ylabel("Count")
plt.title("Reconstruction Error Distribution (Nominal vs Anomaly)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
hist_path = OUT_DIR / "vae_recon_error_hist.png"
plt.savefig(hist_path, dpi=300)
plt.close()
print(f"✅ Saved recon-error histogram to: {hist_path}")

# 4.3 t-SNE of latent space (system-level classes 0..5)
print("\nRunning t-SNE on latent space ...")
tsne = TSNE(
    n_components=2,
    perplexity=min(30, max(5, N // 3)),
    random_state=42,
    init="pca",
    learning_rate="auto",
)
z_2d = tsne.fit_transform(z_mu)

system_names = {
    0: "Nominal",
    1: "CO₂ Leak",
    2: "Valve Stiction",
    3: "Vacuum Anomaly",
    4: "CDRA Degradation",
    5: "OGA Degradation",
}

plt.figure(figsize=(6, 5))
for sys_id, name in system_names.items():
    mask = (y_sys == sys_id)
    plt.scatter(
        z_2d[mask, 0],
        z_2d[mask, 1],
        alpha=0.7,
        s=25,
        label=name,
    )

plt.xlabel("t-SNE dim 1")
plt.ylabel("t-SNE dim 2")
plt.title("VAE Latent Space (t-SNE projection)")
plt.legend(markerscale=1.2, fontsize=8)
plt.grid(alpha=0.3)
plt.tight_layout()
tsne_path = OUT_DIR / "vae_latent_tsne.png"
plt.savefig(tsne_path, dpi=300)
plt.close()
print(f"✅ Saved t-SNE latent space plot to: {tsne_path}")

print("\nAnalysis complete.")


 ANALYZING DENSE VAE – ECLSS SYNTHETIC DATASET
Using preprocessed dir : C:\Users\ahasa\data\eclss_preprocessed
Saving figures to       : C:\Users\ahasa\figures

N_test = 165
  Nominal samples: 15
  Anomaly samples: 150

AUC (reconstruction error vs. binary label): 0.8347
Best threshold by Youden's J: 0.32085
  At this threshold: FPR=0.000, TPR=0.660

Confusion matrix at best threshold:
[[15  0]
 [51 99]] 

Classification report:
              precision    recall  f1-score   support

     Nominal       0.23      1.00      0.37        15
     Anomaly       1.00      0.66      0.80       150

    accuracy                           0.69       165
   macro avg       0.61      0.83      0.58       165
weighted avg       0.93      0.69      0.76       165

✅ Saved ROC curve to: C:\Users\ahasa\figures\vae_roc_recon_error.png
✅ Saved recon-error histogram to: C:\Users\ahasa\figures\vae_recon_error_hist.png

Running t-SNE on latent space (this may take a few seconds)...
✅ Saved t-SNE latent spac

In [44]:
import numpy as np
from pathlib import Path
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix

DATA_DIR = Path("data/eclss_preprocessed")

# Load scores and labels
recon = np.load(DATA_DIR / "vae_test_recon_errors.npy")
y_bin = np.load(DATA_DIR / "y_test_all_binary.npy")  # 0=nominal, 1=anomaly

print("AUC:", roc_auc_score(y_bin, recon))

# ROC curve
fpr, tpr, thresholds = roc_curve(y_bin, recon)

# FNR = 1 - TPR
fnr = 1 - tpr

# Find all thresholds where FPR <= 0.1 and FNR <= 0.1
candidates = np.where((fpr <= 0.10) & (fnr <= 0.10))[0]

if len(candidates) == 0:
    print("❌ No threshold exists with FPR <= 10% and FNR <= 10%.")
else:
    # Among candidates, we can choose e.g. the one with the smallest |FPR - FNR|
    idx = candidates[np.argmin(np.abs(fpr[candidates] - fnr[candidates]))]
    best_thr = thresholds[idx]
    print(f"✅ Found feasible threshold: {best_thr:.5f}")
    print(f"   At this threshold: FPR={fpr[idx]:.3f}, FNR={fnr[idx]:.3f}, TPR={tpr[idx]:.3f}")

    # Evaluate confusion matrix and detailed metrics at this threshold
    y_pred = (recon >= best_thr).astype(int)
    cm = confusion_matrix(y_bin, y_pred)
    print("\nConfusion matrix at chosen threshold:")
    print(cm)
    print("\nClassification report:")
    print(classification_report(y_bin, y_pred, target_names=["Nominal", "Anomaly"]))


AUC: 0.8160000000000001
❌ No threshold exists with FPR <= 10% and FNR <= 10%.
